In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
import os
import sys
import json
import joblib
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from WattPredictor.entity.config_entity import EvaluationConfig
from WattPredictor.config.model_config import ModelConfigurationManager
from WattPredictor.utils.feature import feature_store_instance
from WattPredictor.utils.ts_generator import features_and_target
from sklearn.metrics import mean_squared_error, mean_absolute_error,root_mean_squared_error, r2_score
from WattPredictor.utils.helpers import create_directories, save_json
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.logging import logger

class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
        self.feature_store = feature_store_instance()

    def evaluate(self):
        try:
            df, _ = self.feature_store.get_training_data("elec_wx_features_view")
            df = df[['date', 'demand', 'sub_region_code', 'temperature_2m']]
            df.sort_values("date", inplace=True)
            
            train_df, test_df = df[df['date'] < self.config.cutoff_date], df[df['date'] >= self.config.cutoff_date]

            test_x, test_y = features_and_target(test_df, input_seq_len=self.config.input_seq_len, step_size=self.config.step_size)
            test_x.drop(columns=["date"], errors="ignore", inplace=True)

            model_registry = self.feature_store.project.get_model_registry()
            model_name = "wattpredictor_lightgbm"
            
            models = model_registry.get_models(model_name)
            if not models:
                raise CustomException(f"No models found with name '{model_name}'", sys)
            
            latest_model = models[0] 
            
            
            model_dir = latest_model.download()
            model_path = os.path.join(model_dir, "model.joblib")
            model_instance = joblib.load(model_path)

            preds = model_instance.predict(test_x)

            mse = mean_squared_error(test_y, preds)
            mae = mean_absolute_error(test_y, preds)
            rmse = np.sqrt(mse)
            r2 = r2_score(test_y, preds)
            mape = np.mean(np.abs((test_y - preds) / test_y)) * 100 if np.any(test_y != 0) else np.inf
            adjusted_r2 = 1 - (1 - r2) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1] - 1)

            metrics = {
                "mse": mse,
                "mae": mae,
                "rmse": rmse,
                "mape": mape,
                "r2_score": r2,
                "adjusted_r2": adjusted_r2
            }

            create_directories([os.path.dirname(self.config.metrics_path)])
            save_json(self.config.metrics_path, metrics)
            logger.info(f"Saved evaluation metrics at {self.config.metrics_path}")

            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(test_y[:100], label="Actual", color="blue")
            ax.plot(preds[:100], label="Predicted", color="red")
            ax.set_title("Predicted vs Actual (First 100 Points)")
            ax.set_xlabel("Time Step")
            ax.set_ylabel("Electricity Demand")
            ax.legend()

            create_directories([os.path.dirname(self.config.img_path)])
            fig.savefig(self.config.img_path)
            plt.close()
            logger.info(f"Saved prediction plot at {self.config.img_path}")

            self.feature_store.upload_file_safely(self.config.metrics_path, "eval/metrics.json")
            self.feature_store.upload_file_safely(self.config.img_path, "eval/pred_vs_actual.png")

            logger.info("Evaluation results uploaded to Hopsworks dataset storage")

            return metrics

        except Exception as e:
            raise CustomException("Model evaluation failed", e)

In [3]:
try:
    config = ModelConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = Evaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-17 16:49:07,325: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-17 16:49:07,332: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-17 16:49:07,335: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-17 16:49:07,338: INFO: helpers: created directory at: artifacts]
[2025-07-17 16:49:07,340: INFO: helpers: created directory at: artifacts/model_evaluation]
[2025-07-17 16:49:07,345: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-17 16:49:07,349: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-17 16:49:07,352: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-17 16:49:07,357: INFO: external: Initializing external client]
[2025-07-17 16:49:07,359: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-17 16:49:10,183: INFO: python: Python Engine initialized.]

Logged in to project, explor

Generating TS features: 100%|██████████| 11/11 [00:00<00:00, 22.86it/s]


Downloading: 0.000%|          | 0/2154054 elapsed<00:00 remaining<?

[2025-07-17 16:49:32,510: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-17 16:49:32,511: INFO: helpers: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-17 16:49:32,512: INFO: 29897945: Saved evaluation metrics at artifacts\model_evaluation\metrics.json]
[2025-07-17 16:49:32,529: WARNING: warnings: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
]
[2025-07-17 16:49:32,560: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-17 16:49:32,685: INFO: 29897945: Saved prediction plot at artifacts\model_evaluation\pred_vs_actual.png]


Uploading f:\WattPredictor\artifacts\model_evaluation\metrics.json: 0.000%|          | 0/206 elapsed<00:00 rem…

[2025-07-17 16:49:35,898: INFO: feature_store: Uploaded file to Feature Store: eval/metrics.json]


Uploading f:\WattPredictor\artifacts\model_evaluation\pred_vs_actual.png: 0.000%|          | 0/98053 elapsed<0…

[2025-07-17 16:49:39,744: INFO: feature_store: Uploaded file to Feature Store: eval/pred_vs_actual.png]
[2025-07-17 16:49:39,746: INFO: 29897945: Evaluation results uploaded to Hopsworks dataset storage]
